# Heat transfer in a fin with piecewise variation of some parameters

Note that this notebook should be at the root directory of the repository because it is executed by `jupyter execute` which sets the working directory to the one the notebook is in. This unifies the behavior of developing and running this notebook in VSCode and also using it in the data pipeline.

## Setup

In [ ]:
from functools import partial
from pathlib import Path
import pickle

from IPython.display import Math
import dill
from matplotlib import pyplot as plt
import numpy as np
from sympy import (
    Eq,
    FiniteSet,
    Function,
    Piecewise,
    Subs,
    besselsimp,
    dsolve,
    exp,
    lambdify,
    pi,
    separatevars,
    solveset,
    sqrt,
    symbols,
)
from sympy.printing.latex import latex

from boilerdata.models.project import Project
from boilerdata.utils import chdir_to_nearest_git_root


def math_mod(expr, long_frac_ratio=3, **kwargs):
    return Math(latex(expr, long_frac_ratio=3))


def disp(title, *exprs, **kwargs):
    print(f"{title}:")
    display(*(math_mod(expr, **kwargs) for expr in exprs))


def disp_free(title, eqn, **kwargs):
    disp(title, eqn, **kwargs)
    disp("Free symbols", FiniteSet(*eqn.rhs.free_symbols), **kwargs)

def chdir_to_nearest_git_root_and_get_project():
    """Ensure this notebook runs at project root regardless of how it is executed."""
    chdir_to_nearest_git_root()
    return Project.get_project()

proj = chdir_to_nearest_git_root_and_get_project()

# Substitute this instead of zero to avoid division by zero
eps = np.finfo(float).eps


## Symbols

In [ ]:
parameters = symbols(
    """
    h,
    h_w,
    h_a,
    k,
    q''_s,
    r,
    T_inf,
    T_s,
    """
)
(
    h,  # General convection heat transfer coefficient
    h_w,  # Heat transfer coefficient in the water domain
    h_a,  # Heat transfer coefficient in the air domain
    k,  # Thermal conductivity
    q_s,  # (q''_s) Boiling surface heat flux
    r,  # Radius of rod
    T_inf,  # Ambient temperature
    T_s,  # Boiling surface temperature
) = parameters

independent_variables = symbols(
    """
    x,
    """
)
(x,) = independent_variables  # Axial position

intermediate_variables = symbols(
    """
    q''_0,
    q''_w-a,
    T_0,
    T_w-a,
    x_0,
    x_s,
    x_w-a,
    """
)
(
    q_0,  # (q''_0) Heat flux at x_0, for a general initial condition
    q_w_a,  # (q''_w-a) Heat flux between the water and air domains
    T_0,  # Temperature at x_0, for a general initial condition
    T_w_a,  # (T_w-a) Temperature between the water and air domains
    x_0,  # Axial position of the initial condition
    x_s,  # Axial position of the boiling surface
    x_w_a,  # (x_w-a) Axial position between the water and air domains
) = intermediate_variables

valid_python_subs = symbols(
    """
    q_s, x_w_a
    """
)
(
    q_s_repr,
    x_w_a_repr,
) = valid_python_subs

functions = symbols(
    """
    T*,
    T_a,
    T_w,
    T,
    """,
    cls=Function,  # type: ignore
)
(
    T_int,  # (T*) The general solution to the ODE
    T_a,  # Solution in air
    T_w,  # Solution in water
    T,  # The piecewise combination of the two above solutions
) = functions

for key, val in {
    "Parameters": parameters,
    "Independent variables": independent_variables,
    "Intermediate variables": intermediate_variables,
    "Valid Python substitutions": valid_python_subs,
}.items():
    disp(key, FiniteSet(*val))

disp("Functions", FiniteSet(*(fun(x) for fun in functions)))


## General ODE and its solution

In [ ]:
P = 2 * pi * r
A_c = pi * r**2

ode = Eq(
    T(x).diff(x, 2) - h * P / k / A_c * (T(x) - T_inf),
    0,
)
ics = {
    T(x_0): T_0,
    Subs(T(x).diff(x), x, x_0): q_0 / k,
}
disp("ODE", ode)
disp("Initial conditions", *(Eq(lhs, rhs) for lhs, rhs in ics.items()))


In [ ]:
T_int_expr = dsolve(ode, T(x), ics=ics).rhs  # type: ignore
disp_free("General solution to the ODE", Eq(T_int(x), T_int_expr))


In [ ]:
if ode.subs(T(x), T_int_expr).simplify():
    print("The solution to the ODE is verified by substitution.")
else:
    raise RuntimeError("The solution to the ODE is not verified by substitution.")


## Solution in the water domain

In [ ]:
T_w_expr = T_int_expr.subs(
    {
        x_0: 0,
        q_0: q_s,
        T_0: T_s,
        h: h_w,
    }
)

disp_free("Solution in the water domain", Eq(T_w(x), T_w_expr))


## Values at the domain boundary

In [ ]:
T_w_a_expr = T_w_expr.subs(x, x_w_a)
q_w_a_expr = T_w_expr.diff(x).subs(x, x_w_a) * k  # type: ignore

disp_free("Temperature at the domain transition", Eq(T_w_a, T_w_a_expr))
disp_free("Heat flux at the domain transition", Eq(q_w_a, q_w_a_expr))


## Solution in the air domain

In [ ]:
T_a_int_expr = T_int_expr.subs(
    {
        x_0: x_w_a,
        q_0: q_w_a,
        T_0: T_w_a,
        h: h_a,
    }
)
T_a_expr = T_a_int_expr.subs(
    {
        T_w_a: T_w_a_expr,
        q_w_a: q_w_a_expr,
    }
)

disp_free("Solution in the air domain", Eq(T_a(x), T_a_int_expr))
disp_free("Solution in the air domain, with substitutions", Eq(T_a(x), T_a_expr))


## Check the solution

In [ ]:
if not Eq(
    (T_w_expr.subs(x, x_w_a) - T_a_expr.subs(x, x_w_a)).simplify(),  # type: ignore
    0,
):
    raise AssertionError("Temperature discontinuous at domain transition.")

if not Eq(
    (T_w_expr.diff(x).subs(x, x_w_a) - T_a_expr.diff(x).subs(x, x_w_a)).simplify(),  # type: ignore
    0,
):
    raise AssertionError("Temperature gradient discontinuous at domain transition.")

print("Temperature and temperature gradient are continuous at the domain transition.")


## Piecewise temperature distribution

In [ ]:
T_expr = Piecewise(
    (T_w_expr, x < x_w_a),
    (T_a_expr, True),
)

disp_free("Temperature distribution in the rod", Eq(T(x), T_expr))


## Make the model function amenable to curve fitting

In [ ]:
T_expr2 = T_expr.subs({q_s: q_s_repr, x_w_a: x_w_a_repr})
disp_free("Temperature distribution after substitution", Eq(T(x), T_expr2))


In [ ]:
T_expr3 = T_expr2.evalf(  # type: ignore
    subs={
        r: 0.0047625,
        k: 400.0,
        T_inf: 25.0,
        x_w_a_repr: 0.0381,
        h_w: eps,
        h_a: 100,
    }
)
disp_free("Temperature distribution after float evaluation", Eq(T(x), T_expr3))


## Plot an example curve of the model

In [ ]:
model = lambdify((x, T_s, q_s_repr), T_expr3)


def get_partial_model(model):
    return partial(
        model,
        T_s=105,
        q_s=20 * 100**2,  # 20 W/cm^2
    )


example_model = get_partial_model(model)
fig, ax = plt.subplots(layout="constrained")
x_smooth = np.linspace(0, 0.10)
_ = ax.plot(x_smooth, example_model(x_smooth))


## Serialize the model to a file

In [ ]:
model_file = proj.dirs.model_file

pickle = dill.dumps(model)
Path(model_file).write_bytes(pickle)
unpickled_partial_model = get_partial_model(dill.loads(pickle))
if not np.allclose(example_model(x_smooth), unpickled_partial_model(x_smooth)):
    raise AssertionError("The unpickled model is not the same as the original model.")
